## Code Challenge: Solve the Global Alignment Problem.

Input: A match reward, a mismatch penalty, an indel penalty, and two nucleotide strings.  
Output: The maximum alignment score of these strings followed by an alignment achieving this maximum score.

In [22]:
def global_alignment(match_reward, mismatch_penalty, indel_penalty, seq1, seq2):
    """
    Performs global sequence alignment using the Needleman-Wunsch algorithm.

    Args:
        match_reward (int): Reward for a match between characters.
        mismatch_penalty (int): Penalty for a mismatch between characters.
        indel_penalty (int): Penalty for inserting a gap (indel) in the alignment.
        seq1 (str): First sequence to align.
        seq2 (str): Second sequence to align.

    Returns:
        tuple: A tuple containing the alignment score (int), aligned sequence 1 (str), and aligned sequence 2 (str).

    """

    # Initialize the alignment matrix
    rows = len(seq1) + 1
    cols = len(seq2) + 1
    matrix = [[0] * cols for _ in range(rows)]

    # Fill the first row and column with indel penalties
    for i in range(1, rows):
        matrix[i][0] = matrix[i-1][0] - indel_penalty
    for j in range(1, cols):
        matrix[0][j] = matrix[0][j-1] - indel_penalty

    # Fill the rest of the matrix
    for i in range(1, rows):
        for j in range(1, cols):
            if seq1[i-1] == seq2[j-1]:
                score = match_reward
            else:
                score = -mismatch_penalty
            matrix[i][j] = max(matrix[i-1][j-1] + score,  # Diagonal (match/mismatch)
                              matrix[i-1][j] - indel_penalty,  # Up (gap in seq2)
                              matrix[i][j-1] - indel_penalty)  # Left (gap in seq1)

    # Traceback to find the alignment
    alignment_seq1 = ""
    alignment_seq2 = ""
    i, j = rows - 1, cols - 1
    while i > 0 and j > 0:
        if seq1[i-1] == seq2[j-1]:
            score = match_reward
        else:
            score = -mismatch_penalty

        if matrix[i][j] == matrix[i-1][j-1] + score:  # Diagonal
            alignment_seq1 = seq1[i-1] + alignment_seq1
            alignment_seq2 = seq2[j-1] + alignment_seq2
            i -= 1
            j -= 1
        elif matrix[i][j] == matrix[i-1][j] - indel_penalty:  # Up (gap in seq2)
            alignment_seq1 = seq1[i-1] + alignment_seq1
            alignment_seq2 = '-' + alignment_seq2
            i -= 1
        else:  # Left (gap in seq1)
            alignment_seq1 = '-' + alignment_seq1
            alignment_seq2 = seq2[j-1] + alignment_seq2
            j -= 1

    # Add remaining characters if any
    while i > 0:
        alignment_seq1 = seq1[i-1] + alignment_seq1
        alignment_seq2 = '-' + alignment_seq2
        i -= 1
    while j > 0:
        alignment_seq1 = '-' + alignment_seq1
        alignment_seq2 = seq2[j-1] + alignment_seq2
        j -= 1

    # Calculate the alignment score
    alignment_score = matrix[rows-1][cols-1]

    return alignment_score, alignment_seq1, alignment_seq2


In [23]:
# Sample 
match_reward = 1
mismatch_penalty = 1
indel_penalty = 2
seq1 = "GAGA"
seq2 = "GAT"

sample_output = """-1
GAGA
GA-T"""

score, aligned_seq1, aligned_seq2 = global_alignment(match_reward, mismatch_penalty, indel_penalty, seq1, seq2)
result = f'{score}\n{aligned_seq1}\n{aligned_seq2}'

assert result == sample_output


In [24]:
file_name = 'dataset_247_3'
with open(f'./datasets/{file_name}.txt', 'r') as f:
    match_reward, mismatch_penalty, indel_penalty = map(int, f.readline().strip().split())
    seq1 = f.readline().strip()
    seq2 = f.readline().strip()

score, aligned_seq1, aligned_seq2 = global_alignment(match_reward, mismatch_penalty, indel_penalty, seq1, seq2)
result = f'{score}\n{aligned_seq1}\n{aligned_seq2}'

with open(f'./submissions/{file_name}.txt', 'w') as f:
    f.write(str(result))

## Code Challenge: Solve the Local Alignment Problem.

Input: Two protein strings written in the single-letter amino acid alphabet.  
Output: The maximum score of a local alignment of the strings, followed by a local alignment of these strings achieving the maximum score. Use the PAM250 scoring matrix for matches and mismatches as well as the indel penalty σ = 5.

In [25]:
with open('PAM250.txt', 'r') as file:
    file = file.read().split('\n')
    keys = file[0].split()
    PAM250 = {}
    for i in range(1, len(file)):
        temp = file[i].split()
        key2 = temp[0]
        for j, key in enumerate(keys):
            if not key in PAM250:
                PAM250[key] = {key2: int(temp[j + 1])}
            else:
                PAM250[key][key2] = int(temp[j + 1])

In [26]:
def local_alignment(v, w):
    """
    Performs local alignment on strings v and w using the PAM250 scoring matrix.

    Args:
        v (str): String v.
        w (str): String w.

    Returns:
        tuple: A tuple containing the backtrack matrix and the scoring matrix.
    """
    v = '-' + v
    w = '-' + w
    S = [[0 for _ in range(len(w))] for _ in range(len(v))]
    backtrack = [[0 for _ in range(len(w))] for _ in range(len(v))]

    for i in range(1, len(S)):
        S[i][0] = S[i - 1][0] - 5
        backtrack[i][0] = 1

    for j in range(1, len(S[0])):
        S[0][j] = S[0][j - 1] - 5
        backtrack[0][j] = 2

    for i in range(1, len(v)):
        for j in range(1, len(w)):
            diag = S[i - 1][j - 1] + PAM250[v[i]][w[j]]
            down = S[i - 1][j] - 5
            right = S[i][j - 1] - 5
            S[i][j] = max([0, down, right, diag])
            if S[i][j] == 0:
                backtrack[i][j] = 0
            elif S[i][j] == down:
                backtrack[i][j] = 1
            elif S[i][j] == right:
                backtrack[i][j] = 2
            else:
                backtrack[i][j] = 4

    max_score = -1
    for row in S:
        tmp = max(row)
        if tmp > max_score:
            max_score = tmp
            
    return backtrack, S, max_score


def alignment(backtrack, S, v, w):
    """
    Performs alignment based on the backtrack matrix and the scoring matrix.

    Args:
        backtrack (list): The backtrack matrix.
        S (list): The scoring matrix.
        v (str): String v.
        w (str): String w.
    """
    max_val = -1000
    for r, row in enumerate(S):
        for c, val in enumerate(row):
            if S[r][c] > max_val:
                max_val = S[r][c]
                i = r
                j = c

    res_v = ''
    res_w = ''
    while backtrack[i][j] != 0:
        if backtrack[i][j] == 4:
            res_v = v[i - 1] + res_v
            res_w = w[j - 1] + res_w
            i -= 1
            j -= 1
        elif backtrack[i][j] == 2:
            res_v = '-' + res_v
            res_w = w[j - 1] + res_w
            j -= 1
        else:
            res_v = v[i - 1] + res_v
            res_w = '-' + res_w
            i -= 1

    return res_v, res_w

In [27]:
# Sample Input

sample_v = 'MEANLY'
sample_w = 'PENALTY'

sample_output = """15
EANL-Y
ENALTY"""

backtrack, scoring_matrix, max_score = local_alignment(sample_v, sample_w)
res_v, res_w = alignment(backtrack, scoring_matrix, sample_v, sample_w)

result = f"{max_score}\n{res_v}\n{res_w}"
assert result == sample_output

In [28]:
file_name = 'dataset_247_10'
with open(f'./datasets/{file_name}.txt', 'r') as f:
    seq1 = f.readline().strip()
    seq2 = f.readline().strip()

backtrack, scoring_matrix, max_score = local_alignment(seq1, seq2)
res_v, res_w = alignment(backtrack, scoring_matrix, seq1, seq2)
result = f"{max_score}\n{res_v}\n{res_w}"

with open(f'./submissions/{file_name}.txt', 'w') as f:
    f.write(str(result))

## Edit Distance Problem: Find the edit distance between two strings.

Input: Two strings.  
Output: The edit distance between these strings.

In [29]:
def edit_distance(a, b):
    """
    Calculates the edit distance between two strings a and b.

    Args:
        a (str): The first string.
        b (str): The second string.

    Returns:
        int: The edit distance between the two strings.
    """
    la = len(a)
    lb = len(b)
    matrix = [[0] * (lb + 1) for _ in range(la + 1)]

    for i in range(1, la + 1):
        matrix[i][0] = i

    for j in range(1, lb + 1):
        matrix[0][j] = j

    for i in range(1, la + 1):
        for j in range(1, lb + 1):
            str_a = a[i - 1]
            str_b = b[j - 1]
            match = 1
            if str_a == str_b:
                match = 0
            matrix[i][j] = min(
                matrix[i - 1][j] + 1,
                matrix[i][j - 1] + 1,
                matrix[i - 1][j - 1] + match
            )

    return matrix[-1][-1]


In [30]:
sample_a = 'GAGA'
sample_b = 'GAT'
sample_output = 2

assert edit_distance(sample_a, sample_b) == sample_output

In [31]:
file_name = 'dataset_248_3'
with open(f'./datasets/{file_name}.txt', 'r') as f:
    seq1 = f.readline().strip()
    seq2 = f.readline().strip()

result = f"{edit_distance(seq1, seq2)}"

with open(f'./submissions/{file_name}.txt', 'w') as f:
    f.write(str(result))

## Code Challenge: Solve the Fitting Alignment Problem.

Input: Two amino acid strings.  
Output: A highest-scoring fitting alignment between v and w. Use the BLOSUM62 scoring table.

In [32]:
with open('BLOSUM62.txt', 'r') as file:
    file = file.read().split('\n')
    keys = file[0].split()
    BLOSUM62 = {}
    for i in range(1, len(file)):
        temp = file[i].split()
        key2 = temp[0]
        for j, key in enumerate(keys):
            if not key in BLOSUM62:
                BLOSUM62[key] = {key2: int(temp[j + 1])}
            else:
                BLOSUM62[key][key2] = int(temp[j + 1])

In [33]:
import numpy as np

def fitt_backtrack(v, w, sigma, matrix):
    """
    Performs the Fitting Alignment backtrack step.

    Args:
        v (str): The vertical string.
        w (str): The horizontal string.
        sigma (int): The gap penalty.
        matrix (dict): The substitution matrix.

    Returns:
        np.matrix: The backtrack matrix.
    """
    global s, n, m

    n = len(v)
    m = len(w)

    s = np.matrix(np.zeros((n + 1) * (m + 1), dtype=int).reshape((n + 1, m + 1)))
    backtrack = np.matrix(np.zeros((n + 1) * (m + 1), dtype=int).reshape((n + 1, m + 1)))
    s[0] = [i * -sigma for i in range(m + 1)]
    backtrack[0] = [2 for _ in range(m + 1)]

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            s[i, j] = max([s[i - 1, j] - sigma, s[i, j - 1] - sigma, s[i - 1, j - 1] + matrix[v[i - 1]][w[j - 1]]])
            if s[i, j] == s[i - 1, j] - sigma:
                backtrack[i, j] = 1  # 1 == down
            elif s[i, j] == s[i, j - 1] - sigma:
                backtrack[i, j] = 2  # 2 == right
            elif s[i, j] == s[i - 1, j - 1] + matrix[v[i - 1]][w[j - 1]]:
                backtrack[i, j] = 3  # 3 == diagonal

    return backtrack


def fitt_alignment(v, w, i, j, matrix, sigma=1):
    """
    Performs the Fitting Alignment.

    Args:
        v (str): The vertical string.
        w (str): The horizontal string.
        i (int): The starting index for the vertical string.
        j (int): The starting index for the horizontal string.
        sigma (int): The gap penalty.
        matrix (dict): The substitution matrix.
    """
    global lcs_v, lcs_w, score, backtrack

    backtrack = fitt_backtrack(v, w, sigma, matrix)
    max_coords = np.where(s == np.amax(s[:, -1]))
    max_i, max_j = list(zip(max_coords[0], max_coords[1]))[0]
    i = max_i
    j = max_j
    lcs_v = ''
    lcs_w = ''
    score = 0

    if i == 0 or j == 0:
        return

    def global_lcs(v, i, j):
        global lcs_v, lcs_w, score, backtrack

        while i > 0 and j > 0:
            if 1 == backtrack[i, j]:  # backtracks up a single position
                i -= 1
                lcs_v += v[i]
                lcs_w += '-'
                score -= sigma
                continue
            elif 2 == backtrack[i, j]:  # backtracks left a single position
                j -= 1
                lcs_w += w[j]
                lcs_v += '-'
                score -= sigma
                continue
            elif 3 == backtrack[i, j]:  # backtracks diagonally, match added to lcs
                i -= 1
                j -= 1
                lcs_v += v[i]
                lcs_w += w[j]
                score += matrix[v[i]][w[j]]
                continue
            else:
                i = 0
                j = 0

    if 1 == backtrack[i, j]:
        score -= sigma
        lcs_v += v[i - 1]
        lcs_w += '-'
        global_lcs(v, i - 1, j)
    elif 2 == backtrack[i, j]:
        score -= sigma
        lcs_w += w[j - 1]
        lcs_v += '-'
        global_lcs(v, i, j - 1)
    elif 3 == backtrack[i, j]:
        score += matrix[v[i - 1]][w[j - 1]]
        lcs_v += v[i - 1]
        lcs_w += w[j - 1]
        global_lcs(v, i - 1, j - 1)

    # print('Fitting Alignment Score: ' + str(score))
    # print(lcs_v[::-1])
    # print(lcs_w[::-1])
    return str(score), lcs_v[::-1], lcs_w[::-1]


In [34]:
sample_v = 'DISCREPANTLY'
sample_w = 'PATENT'
sample_output = """20
PA--NT
PATENT"""

score, res_v, res_w = fitt_alignment(sample_v, sample_w, len(sample_v), len(sample_w), BLOSUM62)
result = f"{score}\n{res_v}\n{res_w}"
assert result == sample_output

In [35]:
file_name = 'dataset_248_5'
with open(f'./datasets/{file_name}.txt', 'r') as f:
    seq1 = f.readline().strip()
    seq2 = f.readline().strip()

score, res_v, res_w = fitt_alignment(seq1, seq2, len(seq1), len(seq2), BLOSUM62)
result = f"{score}\n{res_v}\n{res_w}"

with open(f'./submissions/{file_name}.txt', 'w') as f:
    f.write(str(result))

## Code Challenge: Solve the Overlap Alignment Problem.

Input: A match reward, a mismatch penalty, an indel penalty, and two nucleotide strings v and w.  
Output: The score of an optimal overlap alignment of v and w, followed by an alignment of a suffix v' of v and a prefix w' of w achieving this maximum score.

In [52]:
import numpy as np

def overlap_backtrack(v, w, match, sigma, mu):
    """
    Perform overlap alignment using the Needleman-Wunsch algorithm.

    Args:
        v (str): The vertical sequence.
        w (str): The horizontal sequence.
        match (int): The score for a match.
        sigma (int): The gap penalty.
        mu (int): The mismatch penalty.

    Returns:
        np.ndarray: The backtrack matrix.
    """
    global s, n, m

    n = len(v)
    m = len(w)

    s = np.matrix(np.zeros((n+1)*(m+1), dtype=int).reshape((n+1, m+1)))
    backtrack = np.matrix(np.zeros((n+1)*(m+1), dtype=int).reshape((n+1, m+1)))
    s[0] = [i*-sigma for i in range(m+1)]
    backtrack[0] = [2 for i in range(m+1)]

    for i in range(1, n+1):
        for j in range(1, m+1):
            s[i, j] = max([s[i - 1, j] - sigma, s[i, j - 1] - sigma, s[i - 1, j - 1] + match if v[i-1] == w[j-1] else s[i-1, j-1] - mu])  # determines optimal path to current node from all preceding nodes
            if s[i, j] == s[i - 1, j] - sigma:
                backtrack[i, j] = 1  # 1 == down
            elif s[i, j] == s[i, j - 1] - sigma:
                backtrack[i, j] = 2  # 2 == right
            elif s[i, j] == s[i - 1, j - 1] + match:
                backtrack[i, j] = 3  # 3 == diagonal match

    return backtrack


def overlap_alignment(v, w, i, j, match, sigma, mu):
    """
    Perform overlap alignment between two sequences using the Needleman-Wunsch algorithm.

    Args:
        v (str): The vertical sequence.
        w (str): The horizontal sequence.
        i (int): The starting index of the vertical sequence.
        j (int): The starting index of the horizontal sequence.
        match (int): The score for a match.
        sigma (int): The gap penalty.
        mu (int): The mismatch penalty.

    Returns:
        None: The function prints the alignment score, aligned vertical sequence, and aligned horizontal sequence.
    """
    global lcs_v, lcs_w, score, backtrack

    backtrack = overlap_backtrack(v, w, match, sigma, mu)
    max_coords = np.where(s == np.amax(s[-1]))
    max_i, max_j = list(zip(max_coords[0], max_coords[1]))[0]
    i = max_i
    j = max_j
    lcs_v = ''
    lcs_w = ''
    score = 0

    if i == 0 or j == 0:
        return

    def global_lcs(v, i, j):
        global lcs_v, lcs_w, score, backtrack

        while i > 0 and j > 0:
            if 1 == backtrack[i, j]: # backtracks up a single position
                i -= 1
                lcs_v += v[i]
                lcs_w += '-'
                score -= sigma
                continue
            elif 2 == backtrack[i, j]: # backtracks left a single position
                j -= 1
                lcs_w += w[j]
                lcs_v += '-'
                score -= sigma
                continue
            elif 3 == backtrack[i, j]: # backtracks diagonally, match added to lcs
                i -= 1
                j -= 1
                lcs_v += v[i]
                lcs_w += w[j]
                score += match
                continue
            else:
                i -= 1
                j -= 1
                lcs_v += v[i]
                lcs_w += w[j]
                score -= mu
                continue

    if 1 == backtrack[i, j]: # determines first backtracking step to take
        score -= sigma
        lcs_v += v[i-1]
        lcs_w += '-'
        global_lcs(v, i - 1, j)
    elif 2 == backtrack[i, j]:
        score -= sigma
        lcs_w += w[j-1]
        lcs_v += '-'
        global_lcs(v, i, j - 1)
    elif 3 == backtrack[i, j]:
        score += match
        lcs_v += v[i-1]
        lcs_w += w[j-1]
        global_lcs(v, i - 1, j - 1)

    # print('Overlap Alignment Score: ' + str(score))
    # print(lcs_v[::-1])
    # print(lcs_w[::-1])
    return str(score), lcs_v[::-1], lcs_w[::-1]

In [53]:
sample_match, sample_mu, sample_sigma = 1, 1, 2
sample_v = 'GAGA'
sample_w = 'GAT'

sample_output = """2
GA
GA"""

score, res_v, res_w = overlap_alignment(sample_v, sample_w, len(sample_v), len(sample_w), sample_match, sample_sigma, sample_mu)
result = f"{score}\n{res_v}\n{res_w}"
assert result == sample_output

In [54]:
file_name = 'dataset_248_7'
with open(f'./datasets/{file_name}.txt', 'r') as f:
    match, mu, sigma = map(int, f.readline().strip().split())
    seq1 = f.readline().strip()
    seq2 = f.readline().strip()

score, res_v, res_w = overlap_alignment(seq1, seq2, len(seq1), len(seq2), match, sigma, mu)
result = f"{score}\n{res_v}\n{res_w}"

with open(f'./submissions/{file_name}.txt', 'w') as f:
    f.write(str(result))